In [1]:
import norfetools as nt
import osmnx as ox
import matplotlib.pyplot as plt
from mpl_toolkits.basemap import Basemap
import shapely.geometry as sg

import numpy as np
import os

from Global import cities
from shapely.geometry import MultiPolygon, Polygon

from numba import jit

In [4]:
import json

my_dict = {
    'item1': 'value1',
    'item2': 'value2',
    'item3': 'value3'
}
print(my_dict)
print (json.dumps(my_dict, indent=4))

{'item1': 'value1', 'item2': 'value2', 'item3': 'value3'}
{
    "item1": "value1",
    "item2": "value2",
    "item3": "value3"
}


In [2]:
def read_shapefile(m, loc):
    # 从高到低检查文件级别
    for level in range(3, 0, -1):
        shapefile_path = f"Data/@MapShape/gadm36_{loc}_shp/gadm36_{loc}_{level}"
        if os.path.exists(shapefile_path + ".shp"):  # 检查 .shp 文件是否存在
            m.readshapefile(shapefile_path, 'states', drawbounds=True)
            # print(f"Loaded shapefile for level {level}")
            break
    else:  # 如果没有找到任何文件
        print(f"No shapefile found for location {loc}")

def plot_locations_with_basemap(locations, city_boundary, city_name):
    # 将locations中的decimal.Decimal转换为float
    locations = [(float(lat), float(lon)) for lat, lon in locations]
    # print(locations)

    # 设置绘制地图的范围
    lon_min, lat_min, lon_max, lat_max = city_boundary.bounds
    lat_min -= 0.1
    lat_max += 0.1
    lon_min -= 0.1
    lon_max += 0.1

    # 创建一个新的matplotlib图和一个子图
    plt.figure(figsize=(3, 3))
    m = Basemap(projection='merc', llcrnrlat=lat_min, urcrnrlat=lat_max,
                llcrnrlon=lon_min, urcrnrlon=lon_max, resolution='i')
    
    loclist = ["CHN", "DEU", "FRA", "NLD", "SGP", "USA"]
    for loc in loclist:
        read_shapefile(m, loc)

    # 绘制地图细节
    m.drawcoastlines()
    m.drawcountries()
    m.fillcontinents(color='#fef0d5', lake_color='#669dbc')
    m.drawmapboundary(fill_color='#669dbc')
    m.drawstates()

    # 检查city_boundary的类型并适当处理
    if isinstance(city_boundary, sg.Polygon):
        polygons = [city_boundary]
    elif isinstance(city_boundary, sg.MultiPolygon):
        polygons = city_boundary.geoms
    else:
        raise TypeError("Unsupported geometry type")

    # 遍历每一个Polygon并绘制
    for polygon in polygons:
        boundary_coords = np.array(polygon.exterior.coords)
        city_boundary_x, city_boundary_y = zip(*[m(lon, lat) for lon, lat in boundary_coords])
        m.plot(city_boundary_x, city_boundary_y, marker=None, color='#c11221', linewidth=1)

    # 计算经纬度刻度的间隔和起始点，确保三个刻度
    def calculate_ticks(start, end, num_ticks=3):
        start = np.round(start + 0.05, 1)  # 向上取整到最接近的0.01
        end = np.round(end - 0.05, 1)      # 向下取整到最接近的0.01
        tick_interval = np.round((end - start) / num_ticks, 2)
        # ticks = np.arange(start, end + tick_interval*2, tick_interval)
        ticks = np.linspace(start, end, num_ticks)
        return ticks

    lon_ticks = calculate_ticks(lon_min, lon_max, 3)
    lat_ticks = calculate_ticks(lat_min, lat_max, 6)

    # 绘制网格线并自定义标签格式，添加南北纬和东西经标记
    meridians = m.drawmeridians(lon_ticks, labels=[0,0,0,1], fmt=lambda x: f'{x:.2f}°E' if x >= 0 else f'{-x:.2f}°W')
    parallels = m.drawparallels(lat_ticks, labels=[1,0,0,0], fmt=lambda x: f'{x:.2f}°N' if x >= 0 else f'{-x:.2f}°S')

    # 转换坐标并在地图上标记
    x, y = zip(*[m(lon, lat) for lat, lon in locations])
    m.scatter(x, y, marker='o', color='#fbb969', zorder=10, s=1, label="Sampling Points")

    plt.legend(loc='upper left', fontsize=8)
    
    plt.title(f"Map of {city_name}")
    

In [3]:
def extract_largest_polygon(gdf):
    """
    从GeoDataFrame中提取面积最大的多边形。

    参数:
    gdf -- GeoDataFrame，其中包含多个几何对象。

    返回:
    largest_polygon -- 面积最大的多边形对象。
    """
    max_area = 0
    largest_polygon = None

    for geom in gdf['geometry']:
        # 如果是MultiPolygon，遍历所有子多边形
        if isinstance(geom, MultiPolygon):
            for poly in geom:
                area = poly.area
                if area > max_area:
                    max_area = area
                    largest_polygon = poly
        # 如果是单个Polygon，直接比较面积
        elif isinstance(geom, Polygon) and geom.area > max_area:
            max_area = geom.area
            largest_polygon = geom
        else:
            continue  # 其他几何类型则忽略

    return largest_polygon

In [4]:
from shapely.geometry import Point
from shapely.ops import transform
from pyproj import CRS, Transformer

# Create the transformer to convert from EPSG:3857 to EPSG:4326
transformer = Transformer.from_crs('epsg:3857', 'epsg:4326', always_xy=True)

def generate_uniform_points_in_polygon(polygon, num_points):
    """
    Generate a grid of uniformly distributed points within the given polygon.
    
    Parameters:
    polygon -- shapely.geometry.Polygon object.
    num_points -- the grid size for the number of points to generate.
    
    Returns:
    points -- list of points' (longitude, latitude) within the polygon.
    """
    minx, miny, maxx, maxy = polygon.bounds
    # print(minx, miny, maxx, maxy)

    # Generate points in the grid
    points = []
    for i in np.linspace(minx, maxx, num_points):
        for j in np.linspace(miny, maxy, num_points):
            point = Point(i, j)
            if polygon.contains(point):
                # Transform the point to longitude, latitude
                # lon, lat = transformer.transform(i, j)
                points.append((j, i))

    return points


def extract_largest_polygon_from_boundary(boundary):
    """
    从行政区边界中提取最大的多边形。

    参数:
    boundary -- 可能是Polygon或MultiPolygon对象的行政区边界。

    返回:
    largest_polygon -- 面积最大的多边形对象。
    """
    # 如果boundary本身是一个多边形，直接返回它
    if isinstance(boundary, Polygon):
        return boundary
    
    # 如果boundary是一个多多边形对象，找出其中面积最大的
    elif isinstance(boundary, MultiPolygon):
        max_area = 0
        largest_polygon = None
        for polygon in boundary.geoms:
            area = polygon.area
            if area > max_area:
                max_area = area
                largest_polygon = polygon
        return largest_polygon

    else:
        raise TypeError("The boundary must be a Polygon or MultiPolygon object")


In [5]:
import os

def list_folders_cleaned(directory):
    # 获取指定路径下的所有项目
    items = os.listdir(directory)
    # 过滤出目录，忽略文件
    folders = [item for item in items if os.path.isdir(os.path.join(directory, item))]
    # 删除文件夹名中的"-POI"
    cleaned_folders = [folder.replace('-GD', '') for folder in folders]
    return cleaned_folders

def rename_folders(directory):
    # 遍历指定目录下的所有项目
    for item in os.listdir(directory):
        item_path = os.path.join(directory, item)
        # 确认这是一个目录
        if os.path.isdir(item_path):
            # 检查目录名是否含有"-POI"
            if '-POI' in item:
                # 生成新的文件夹名称，将"-POI"替换为"-GD"
                new_name = item.replace('-POI', '-GD')
                new_path = os.path.join(directory, new_name)
                # 重命名文件夹
                os.rename(item_path, new_path)
                print(f'Renamed "{item}" to "{new_name}"')


# 使用函数
directory_path = 'Y:\PublicData\City-POI'  # 请替换为你的文件夹路径
# rename_folders(directory_path)
China_cities = list_folders_cleaned(directory_path)
print(China_cities)

['Beijing', 'Changsha', 'Chengdu', 'Chongqing', 'Dongguan', 'Foshan', 'Guangzhou', 'Hangzhou', 'Nanjing', 'Ningbo', 'Qingdao', 'Shanghai', 'Shenzhen', 'Suzhou', 'Tianjin', 'Wenzhou', 'Wuhan', 'Wuxi', 'Xian', 'Zhengzhou']


In [6]:
for city_name in China_cities:
    # 设置OSMnx的配置（可选，例如代理服务器的设置）
    # ox.config(use_cache=True)
    ox.settings.use_cache = True

    # 获取行政区域的边界
    place_name = city_name  # 你可以更改为你需要的城市
    city = ox.geocode_to_gdf(place_name)
    city_boundary = city.unary_union
    largest_polygon = extract_largest_polygon_from_boundary(city_boundary)
    generated_points = generate_uniform_points_in_polygon(largest_polygon, 20)

    nt.Save_data([largest_polygon, generated_points], "Data/@boundary/%s.pkl"%city_name)
    plot_locations_with_basemap(generated_points, largest_polygon, city_name)

    nt.SaveFig(1, "%s.png"%city_name, "Figure/Boundary/")